---
## Importación de librerías y configuración de display()

En esta parte importamos las librerías que usamos y cambiamos la configuración para tener un display/print correctos/útiles

In [ ]:
# Importación de librerias:
import numpy as np
import pandas as pd 
import seaborn as sns

In [748]:
# Configuraciones de display:
pd.set_option("display.max_columns", 50)   # Limita la visualización a max 50 columnas. Si se excede genera "...".NO existe display.min_columns
pd.set_option("display.max_rows", 200)     # Limita la visualización a max 20 columnas. Si se excede genera "..."
pd.set_option("display.min_rows", 10)      # Una vez que display.max_rows se excede, la display.min_rows determina cuántas filas se muestran.
#pd.reset_option("^display")               # Este comando resetea todas las opciones por defecto.

---
### Importación del archivo **properati.csv**

In [805]:
# Importación del archivo de Properati:
data = pd.read_csv("properati.csv", sep = ",", low_memory=False) # low_memory=False es para que lea el archivo con la mayor cantidad de memoria dsponible y no infiera "mal" un tipo de dato.

In [806]:
# Muestra de la tabla:
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
42088,42088,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9495707333,-60.6731097363",-32.949571,-60.673110,870000.0,ARS,860924.38,48792.79,31.0,31.0,1573.960968,28064.516129,NaN,NaN,NaN,http://www.properati.com.ar/18vpw_venta_depart...,El edificio se encuentra ubicado en un nodo de...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/1/ABzJtHhJTDmW7r...
80260,80260,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,NaN,NaN,NaN,59000.0,USD,1041025.50,59000.00,NaN,35.0,NaN,1685.714286,NaN,NaN,NaN,http://www.properati.com.ar/1awm4_venta_depart...,"Mono ambiente de 35 m2, A Estrenar, Entrega di...",Mono ambiente en La Plata,https://thumbs4.properati.com/3/KO6MNGbWQLqH7c...
118877,118877,sell,house,Olivos,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430310.0,NaN,NaN,NaN,260000.0,USD,4587570.00,260000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cfxo_venta_casa_o...,CLAUDIO PARRA PROPIEDADESCASA CENTRAL: Av. Fle...,Casa - Olivos-Maipu/Uzal,https://thumbs4.properati.com/0/w1Mxwusk0CY9jA...
66331,66331,sell,house,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|,Argentina,Bs.As. G.B.A. Zona Norte,3427753.0,NaN,NaN,NaN,580000.0,USD,10233810.00,580000.00,NaN,310.0,NaN,1870.967742,NaN,NaN,NaN,http://www.properati.com.ar/1adna_venta_casa_t...,Contacto: 15- | 15-Excelente asa a la Laguna e...,Espectacular Casa a la LAGUNA Santa Catalina,https://thumbs4.properati.com/5/jzOv-6nZjgfqqZ...
39590,39590,sell,PH,Boca,|Argentina|Capital Federal|Boca|,Argentina,Capital Federal,3436004.0,"-34.6322548303,-58.3605471429",-34.632255,-58.360547,115000.0,USD,2029117.50,115000.00,86.0,NaN,1337.209302,NaN,2.0,3.0,NaN,http://www.properati.com.ar/18onk_venta_depart...,Venta de Departamento tipo PH 3 y 1/2 AMBIENTE...,PH EN VENTA,https://thumbs4.properati.com/7/qyBWZ4B-bOCXVY...


---
### Inspección de la tabla sin limpieza previa

Se realiza un análisis básico para entender los datos e información que almacena la tabla

In [808]:
# Dimensiones:
print("Dimensiones de la tabla: ", data.shape)

Dimensiones de la tabla:  (121220, 26)


In [810]:
# Tipo de datos de las columnas:
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

Analizando las columnas una a una podemos ver que los tipos de datos son los correctos segun muestra el df.

In [811]:
# Nombre de las columnas:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [813]:
# Para evitar inconvenientes en el análisis pasamos todos los strings a uppercase():
data = data.applymap(lambda x: x.upper() if type(x) == str else x)
display(data.sample(10))

# Revisamos que no se alteraron los tipos de datos:
data.dtypes

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
11223,11223,SELL,HOUSE,PALERMO SOHO,|ARGENTINA|CAPITAL FEDERAL|PALERMO|PALERMO SOHO|,ARGENTINA,CAPITAL FEDERAL,3430234.0,NaN,NaN,NaN,500000.0,USD,8822250.00,500000.00,211.0,NaN,2369.668246,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/16ATT_VENTA_TERREN...,"TERRENO PROPIO DE 211M2 FRENTE 8,70 POR 24,20 ...",CASA EN PLENO PALERMO SOHO 2 UNIDADES EN BLOCK...,HTTPS://THUMBS4.PROPERATI.COM/1/CD8QFFKAKQWG-M...
100071,100071,SELL,HOUSE,MORÓN,|ARGENTINA|BS.AS. G.B.A. ZONA OESTE|MORÓN|MORÓN|,ARGENTINA,BS.AS. G.B.A. ZONA OESTE,3430544.0,NaN,NaN,NaN,260000.0,USD,4587570.00,260000.00,500.0,170.0,520.000000,1529.411765,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1BNF8_VENTA_CASA_M...,VENTA CASA EN CASTELAR CON PISCINAMUY BUENA PR...,CASA EN MORON,HTTPS://THUMBS4.PROPERATI.COM/1/6ZDWTVZIZOQMSB...
32051,32051,SELL,APARTMENT,POSADAS,|ARGENTINA|MISIONES|POSADAS|,ARGENTINA,MISIONES,3429886.0,NaN,NaN,NaN,650000.0,ARS,643219.30,36454.38,NaN,NaN,NaN,NaN,NaN,1.0,NaN,HTTP://WWW.PROPERATI.COM.AR/185S6_VENTA_DEPART...,"ESPACIOSO DEPARTAMENTO EN CHACRA 32 33 , ZONA ...",ESPACIOSO DEPARTAMENTO 3 DORM CH 32 33 ZONA BU...,HTTPS://THUMBS4.PROPERATI.COM/0/VIHS-07JGZVBMN...
85372,85372,SELL,HOUSE,TREVELÍN,|ARGENTINA|CHUBUT|TREVELÍN|,ARGENTINA,CHUBUT,3833675.0,NaN,NaN,NaN,420000.0,USD,7410690.00,420000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1B2XM_VENTA_CASA_T...,"LOTE UBICADO A 500 MTS DE LA RUTA 71, CON UNA ...",DESARROLLO TURISTICO,HTTPS://THUMBS4.PROPERATI.COM/3/B_R8RQJRSNCUIV...
29728,29728,SELL,APARTMENT,VILLA CRESPO,|ARGENTINA|CAPITAL FEDERAL|VILLA CRESPO|,ARGENTINA,CAPITAL FEDERAL,3427458.0,"-34.5952512,-58.4407652",-34.595251,-58.440765,140977.0,USD,2487468.67,140977.00,45.0,39.0,3132.822222,3614.794872,9.0,NaN,2200.0,HTTP://WWW.PROPERATI.COM.AR/17ZQU_VENTA_DEPART...,VENTA DEPARTAMENTO 2 AMBIENTES VILLA CRESPOEDI...,DEPARTAMENTO EN VENTA,HTTPS://THUMBS4.PROPERATI.COM/0/8IJNBGH7CKAGT5...
74855,74855,SELL,HOUSE,LINIERS,|ARGENTINA|CAPITAL FEDERAL|LINIERS|,ARGENTINA,CAPITAL FEDERAL,3431333.0,NaN,NaN,NaN,800000.0,USD,14115600.00,800000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1APJA_VENTA_CASA_L...,VENTA CASA 5 AMBIENTES LINIERS. A METROS DE AV...,VENTA CASA LINIERS. UNICA EN LA ZONA!,HTTPS://THUMBS4.PROPERATI.COM/4/QE6AJO3A1YZY7B...
2073,2073,SELL,HOUSE,TIGRE,|ARGENTINA|BS.AS. G.B.A. ZONA NORTE|TIGRE|,ARGENTINA,BS.AS. G.B.A. ZONA NORTE,3427753.0,"-34.36105728,-58.68397903",-34.361057,-58.683979,NaN,NaN,NaN,NaN,335.0,256.0,NaN,NaN,NaN,3.0,NaN,HTTP://WWW.PROPERATI.COM.AR/15JS2_VENTA_CASA_T...,HERMOSA CASA A LA LAGUNA EN EL BARRIO SAN GABR...,U$D 497.000 - CASA EN VENTA -,HTTPS://THUMBS4.PROPERATI.COM/8/QADKXFUD5IIIJH...
77797,77797,SELL,HOUSE,TEMPERLEY,|ARGENTINA|BS.AS. G.B.A. ZONA SUR|LOMAS DE ZAM...,ARGENTINA,BS.AS. G.B.A. ZONA SUR,3427776.0,"-34.7620906,-58.3834387",-34.762091,-58.383439,280000.0,USD,4940460.00,280000.00,NaN,268.0,NaN,1044.776119,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1ATAO_VENTA_CASA_T...,EXCELENTE CHALET EN 2 PLANTAS. REJAS AL FRENT...,SEANZ PEÑA 1545,HTTPS://THUMBS4.PROPERATI.COM/0/QFCJKZTWZDCJOM...
42727,42727,SELL,APARTMENT,FLORES,|ARGENTINA|CAPITAL FEDERAL|FLORES|,ARGENTINA,CAPITAL FEDERAL,3433918.0,"-34.6300687972,-58.4700423386",-34.630069,-58.470042,160000.0,USD,2823120.00,160000.00,143.0,135.0,1118.881119,1185.185185,NaN,NaN,3200.0,HTTP://WWW.PROPERATI.COM.AR/18X1M_VENTA_DEPART...,CODIGO: 3410-201 UBICADO EN: NAZCA 100 - PUBL...,OPORTUNIDAD – DEPARTAMENTO EN VENTA EN FLORES ...,HTTPS://THUMBS4.PROPERATI.COM/3/BUVZALSGQS5VET...
31698,31698,SELL,HOUSE,MUÑIZ,|ARGENTINA|BS.AS. G.B.A. ZONA NORTE|SAN MIGUEL...,ARGENTINA,BS.AS. G.B.A. ZONA NORTE,3430513.0,"-34.550317257,-58.705899073",-

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [814]:
# Porcentaje de nulos por columnas:
(data.isnull().sum()/data.shape[0]*100).round(2)

Unnamed: 0                     0.00
operation                      0.00
property_type                  0.00
place_name                     0.02
place_with_parent_names        0.00
country_name                   0.00
state_name                     0.00
geonames_id                   15.44
lat-lon                       42.53
lat                           42.53
lon                           42.53
price                         16.84
currency                      16.84
price_aprox_local_currency    16.84
price_aprox_usd               16.84
surface_total_in_m2           32.44
surface_covered_in_m2         16.42
price_usd_per_m2              43.39
price_per_m2                  27.69
floor                         93.48
rooms                         60.91
expenses                      88.23
properati_url                  0.00
description                    0.00
title                          0.00
image_thumbnail                2.57
dtype: float64

In [815]:
# Información general:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [816]:
# De las columnas numéricas podemos obtener ciertas estadísticas, excluyendo los valores NaN:
data.describe().round(2)

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.00,102503.00,69670.00,69670.00,1.008100e+05,1.008100e+05,100810.00,81892.00,101313.00,68617.00,87658.00,7899.00,47390.00,14262.00
mean,60609.50,3574442.32,-34.63,-59.27,4.685259e+05,4.229397e+06,239700.58,233.80,133.05,2160.09,6912.22,17.45,3.08,5009.23
std,34993.34,354130.62,1.98,2.30,2.260101e+06,6.904714e+06,391323.88,1782.22,724.35,2759.29,28378.64,120.24,1.86,120440.26
min,0.00,3427208.00,-54.82,-75.68,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.60,1.51,1.00,1.00,1.00
25%,30304.75,3430234.00,-34.67,-58.73,1.100000e+05,1.583309e+06,89733.88,50.00,45.00,1218.18,1550.00,1.00,2.00,1000.00
50%,60609.50,3433910.00,-34.60,-58.48,1.850000e+05,2.558452e+06,145000.00,84.00,75.00,1800.00,2213.11,3.00,3.00,2000.00
75%,90914.25,3836668.00,-34.44,-58.40,4.200000e+05,4.675792e+06,265000.00,200.00,150.00,2486.41,3355.55,6.00,4.00,4000.00
max,121219.00,6948895.00,4.55,-53.73,6.500000e+08,8.212711e+08,46545445.00,200000.00,187000.00,206333.33,3999999.96,3150.00,32.00,10001500.00


In [671]:
# De las columnas NO numéricas (categóricas) podemos obtener ciertas estadísticas también:
data.describe(include=['object', 'bool'])      # Se especifica el tipo de dato a analizar.

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,lat-lon,currency,properati_url,description,title,image_thumbnail
count,121220,121220,121197,121220,121220,121220,69670,100809,121220,121218,121220,118108
unique,1,4,1060,1164,1,28,47203,4,121220,104004,71654,114389
top,SELL,APARTMENT,CÓRDOBA,|ARGENTINA|SANTA FE|ROSARIO|,ARGENTINA,CAPITAL FEDERAL,"-34.4026444,-58.6684776",USD,HTTP://WWW.PROPERATI.COM.AR/17Y7K_VENTA_CASA_M...,AVISO LEGAL: LAS DESCRIPCIONES ARQUITECTÓNICAS...,DEPARTAMENTO EN VENTA,HTTPS://THUMBS4.PROPERATI.COM/5/YYMIU8BHQI9KXC...
freq,121220,71065,9254,8504,121220,32316,312,87587,1,358,5025,255


---
### Creación de función general para el análisis de columnas y sus valores

Creamos una función que nos permite analizar rápidamente las columnas (valores nulos y no nulos) y otra que nos permte analizar los valores dentro de esa columna (value_counts)

#### Función my_info_columnas()

In [819]:
# Creamos una función que nos devuelve info de sus columnas. Recibe el dataframe y un argumento que indica si analizamos solo las columnas con algún valor NaN o todas.
def my_info_columnas(data, solo_nan='si'):
    data_null = data.isnull().sum()                                                                               # Df de elementos nulos.
    data_null_porc = (100*data_null/len(data)).round(2)                                                           # Df de elementos nulos %.
    data_null_concat = pd.concat([data_null, data_null_porc], axis=1)                                             # Concatena los df anteriores.
    data_null_ok = data_null_concat.rename(columns = {0 : 'Cantidad de NaN', 1 : '% de NaN'})                     # Renombra las columnas del df anterior.
    data_null_ok['Dtype'] = data.dtypes                                                                           # Pego el tipo de dato al df anterior.
    
    dict_unique_size = {}                                                                                         # Creao diccionario vacio.
    for col in data:                                                                                              # Quiero pegarle los valores únicos de cada columna.
        dict_unique_size[col] = data[col].unique().size                                                           # key = nombre de col, values = cant de unicos.
    
    serie_unique_size = pd.Series(dict_unique_size)                                                               # Convierto del diccionario a serie.
    data_null_ok['Valores Unicos'] = serie_unique_size                                                            # Lo agrego como otra columna.

    if solo_nan == 'si':                                                                                           # Creo if para ver si analizo todas las columnas o solo las NaN.
        data_null_ok = data_null_ok[data_null_ok.iloc[:,0] != 0].sort_values('% de NaN', ascending=False)          # Filtra y ordeno las columnas con algún NaN.
    else:
         data_null_ok = data_null_ok.sort_values(by = ['% de NaN','Cantidad de NaN'] , ascending = [False, False]) # Filtra y ordeno las columnas con algún NaN.
    
    print("Tu dataframe tiene:")
    print(str((data.isnull().sum().sum()/data.size*100).round(2)) + "% de valores NaN en total.")
    print(str(data.shape[0]) + " filas")
    
    if solo_nan == 'si':
        print(str(data.shape[1]) + " columnas de las cuales " + str(data_null_ok.shape[0]) + " tienen valores NaN:")
    else:
        print(str(data.shape[1]) + " columnas:")

    display(data_null_ok)
    return

In [821]:
my_info_columnas(data, solo_nan='si')      # Llamamos a nuestra función. Para analizar TODAS las columnas pasamos nuestro df + solo_nan='no'

Tu dataframe tiene:
22.14% de valores NaN en total.
121220 filas
26 columnas de las cuales 18 tienen valores NaN:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
floor,113321,93.48,float64,183
expenses,106958,88.23,float64,983
rooms,73830,60.91,float64,32
price_usd_per_m2,52603,43.39,float64,25568
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
surface_total_in_m2,39328,32.44,float64,1688
price_per_m2,33562,27.69,float64,25186
price,20410,16.84,float64,9747


In [822]:
my_info_columnas(data, solo_nan='no')      # Llamamos a nuestra función. Para analizar TODAS las columnas pasamos nuestro df + solo_nan='no'

Tu dataframe tiene:
22.14% de valores NaN en total.
121220 filas
26 columnas:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
floor,113321,93.48,float64,183
expenses,106958,88.23,float64,983
rooms,73830,60.91,float64,32
price_usd_per_m2,52603,43.39,float64,25568
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
surface_total_in_m2,39328,32.44,float64,1688
price_per_m2,33562,27.69,float64,25186
currency,20411,16.84,object,5


#### Función my_columna_valores()

In [674]:
# Creamos una función para analizar los valores de las diferentes columnas:
def my_columna_valores(data, col='todas'):                                                                       # Recibe el df y la columna a analizar.
    if col =='todas':
        for col in data:
            print("\nColumna: " + col + " ----------------------------------------------------\n")
            print("Cantidad de elementos únicos: ", data[col].unique().size)
            print(data[col].value_counts(dropna=False))
    else:
        print("Columna: " + col)
        print("Cantidad de elementos únicos: ", data[col].unique().size)
        print(data[col].value_counts(dropna=False))
    return

In [824]:
my_columna_valores(data, col='currency')        # Analizamos los valores de cada columna. Se le pasa el df + nombre de la columna. Si queremos todas, pasar 'todas'

Columna: currency
Cantidad de elementos únicos:  5
USD    87587
NaN    20411
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64


In [825]:
my_columna_valores(data, col='todas')


Columna: Unnamed: 0 ----------------------------------------------------

Cantidad de elementos únicos:  121220
2047     1
54576    1
29988    1
32037    1
25894    1
        ..
2772     1
725      1
6870     1
4823     1
0        1
Name: Unnamed: 0, Length: 121220, dtype: int64

Columna: operation ----------------------------------------------------

Cantidad de elementos únicos:  1
SELL    121220
Name: operation, dtype: int64

Columna: property_type ----------------------------------------------------

Cantidad de elementos únicos:  4
APARTMENT    71065
HOUSE        40268
PH            5751
STORE         4136
Name: property_type, dtype: int64

Columna: place_name ----------------------------------------------------

Cantidad de elementos únicos:  1061
CÓRDOBA                        9254
ROSARIO                        8504
MAR DEL PLATA                  6534
TIGRE                          3324
NORDELTA                       3315
                               ... 
RUFINO             

---
### Eliminación de columnas redundantes

Realizamos esta eliminación de columnas para poder optimizar el espacio de trabajo así como una mejor visualización de los datos necesarios
Por ejemplo la columna country_name siempre tiene el valor Argentina.

In [826]:
# Observando el dataframe notamos que Unnamed:  0 es igual al índice así que lo eliminamos.
data['Unnamed: 0']

0              0
1              1
2              2
3              3
4              4
           ...  
121215    121215
121216    121216
121217    121217
121218    121218
121219    121219
Name: Unnamed: 0, Length: 121220, dtype: int64

In [827]:
data.drop('Unnamed: 0', axis = 'columns', inplace=True)
data.columns

In [828]:
# Vemos que operation y country_name toma un único valor para todo el df. Veamos cuales son:
my_columna_valores(data, col='operation')
my_columna_valores(data, col='country_name')

Columna: operation
Cantidad de elementos únicos:  1
SELL    121220
Name: operation, dtype: int64
Columna: country_name
Cantidad de elementos únicos:  1
ARGENTINA    121220
Name: country_name, dtype: int64


In [ ]:
# Borremos la columna ya que son redundantes por lo menos por ahora:
data.drop(['operation', 'country_name'], axis = 'columns', inplace=True)
data.columns

#### Análisis de columnas lat-lon, lat y lon

El objetivo es eliminar columnas redundantes comparando si la información es la misma.

In [831]:
# Analicemos las columnas lat-lon con lat y lon por separado. Puede haber información redundante. Podemos prescindir de algunas columnas:
data_lat_lon = data[['lat-lon', 'lat', 'lon']]   # Nuevo df solo con estas columnas para su análisis.
data_lat_lon

,lat-lon,lat,lon
0,"-34.6618237,-58.5088387",-34.661824,-58.508839
1,"-34.9038831,-57.9643295",-34.903883,-57.964330
2,"-34.6522615,-58.5229825",-34.652262,-58.522982
3,"-34.6477969,-58.5164244",-34.647797,-58.516424
4,"-38.0026256,-57.5494468",-38.002626,-57.549447
...,...,...,...
121215,NaN,NaN,NaN
121216,NaN,NaN,NaN
121217,"-34.5706388726,-58.4755963355",-34.570639,-58.475596
121218,NaN,NaN,NaN


In [832]:
# Analizo sus valores nulos a ver si son los mismos:
my_info_columnas(data_lat_lon)

Tu dataframe tiene:
42.53% de valores NaN en total.
121220 filas
3 columnas de las cuales 3 tienen valores NaN:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523


Vemos que lat y lon están redondeados pero son los mismos. Tienen la misma cantidad de elementos nulos.
Tenemos dudas de los valores únicos, si serán los mismos por separado. Por lo tanto los separamos en columnas lat-lon y comparamos valores únicos:

In [833]:
data_lat_lon_str = data_lat_lon['lat-lon'].str.split(',',expand=True).rename(columns = {0 : 'lat_split', 1 : 'lon_split'})  # Será de tipo string las columnas.
data_lat_lon_str

,lat_split,lon_split
0,-34.6618237,-58.5088387
1,-34.9038831,-57.9643295
2,-34.6522615,-58.5229825
3,-34.6477969,-58.5164244
4,-38.0026256,-57.5494468
...,...,...
121215,NaN,NaN
121216,NaN,NaN
121217,-34.5706388726,-58.4755963355
121218,NaN,NaN


In [835]:
# Cambiaoms el tipo de dato porque es mas conveniente es float.
data_lat_lon_float = data_lat_lon_str.astype('float64')

# Agrego las columnas separadas al df inicial
data_lat_lon[['lat_split','lon_split']] = data_lat_lon_float[['lat_split','lon_split']]

# Vemos como quedó:
data_lat_lon

In [838]:
# Analizamos nuevamente la info de las columnas:
my_info_columnas(data_lat_lon)

Tu dataframe tiene:
42.53% de valores NaN en total.
121220 filas
5 columnas de las cuales 5 tienen valores NaN:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
lat_split,51550,42.53,float64,46496
lon_split,51550,42.53,float64,46523


Vemos que los valores faltantes y únicos de lat/lon_split son los mismos que del df original por lo que estamos seguros que la información de lat-lon es redundante. Eliminamos entonces la columna lat-lon del df de properati:

In [857]:
# Eliminamos la columna ya que vimos que lat y lon por separado tienen la misma información:
data.drop('lat-lon', axis = 'columns', inplace=True)

### Análisis de variables mas importantes

Realizamos una visualización de los valores de las variables mas importantes. Usamos nuestra función creada para evaluar los valores my_columna_valores()

In [858]:
data.sample(5)

,property_type,place_name,place_with_parent_names,state_name,geonames_id,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
52964,APARTMENT,MAR DEL PLATA,|ARGENTINA|BUENOS AIRES COSTA ATLÁNTICA|MAR DE...,BUENOS AIRES COSTA ATLÁNTICA,3430863.0,-38.026436,-57.533525,138000.0,USD,2434941.0,138000.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,HTTP://WWW.PROPERATI.COM.AR/19KA5_VENTA_DEPART...,HERMOSO DEPTO DE 3 AMBIENTES A LA CALLE . LIVI...,3 AMBIENTES - PLAYA GRANDE,HTTPS://THUMBS4.PROPERATI.COM/0/YDY1JKJVY6VCG3...
20161,APARTMENT,MAR DEL PLATA,|ARGENTINA|BUENOS AIRES COSTA ATLÁNTICA|MAR DE...,BUENOS AIRES COSTA ATLÁNTICA,3430863.0,-38.022759,-57.529789,NaN,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,4.0,NaN,HTTP://WWW.PROPERATI.COM.AR/17BRC_VENTA_DEPART...,EXCELENTE PISO CON VISTA AL PARQUE SAN MARTÍN ...,VENTA PISO EN PLAYA GRANDE 240 MTS - 3 COCHERA...,HTTPS://THUMBS4.PROPERATI.COM/9/KJS_JF78LMOKJW...
5262,HOUSE,CAPITAL FEDERAL,|ARGENTINA|CAPITAL FEDERAL|,CAPITAL FEDERAL,3433955.0,NaN,NaN,495000.0,USD,8734027.5,495000.0,1003.0,375.0,493.519442,1320.000000,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/15PMS_VENTA_CASA_B...,"RECLUS ESQUINA R. L. FALCON, CON VIGILANCIA. B...","CASA 1003M² CON COCHERA EN RECLUS, ELISEO 2500...",HTTPS://THUMBS4.PROPERATI.COM/4/OGCMY_6DHPNNRB...
1582,APARTMENT,MAR DEL PLATA,|ARGENTINA|BUENOS AIRES COSTA ATLÁNTICA|MAR DE...,BUENOS AIRES COSTA ATLÁNTICA,3430863.0,-38.003875,-57.546874,43000.0,USD,758713.5,43000.0,20.0,NaN,2150.000000,NaN,NaN,1.0,NaN,HTTP://WWW.PROPERATI.COM.AR/15IS5_VENTA_DEPART...,AMBIENTE CÓMODO AL LATERAL AMBIENTE CÓMODO AL ...,1 AMBIENTE LATERAL,HTTPS://THUMBS4.PROPERATI.COM/7/061ZJ94RELGXDO...
36309,HOUSE,BARRIO CERRADO SAN ANDRES,|ARGENTINA|BS.AS. G.B.A. ZONA NORTE|TIGRE|BARR...,BS.AS. G.B.A. ZONA NORTE,NaN,-34.384760,-58.692851,320000.0,USD,5646240.0,320000.0,NaN,220.0,NaN,1454.545455,NaN,6.0,NaN,HTTP://WWW.PROPERATI.COM.AR/18GKJ_VENTA_CASA_T...,CORREDOR RESPONSABLE: MATIAS CASTELLI - CUCICB...,CASA EN VENTA APTO CRÉDITO EN BARRIO SAN ANDRÉS,HTTPS://THUMBS4.PROPERATI.COM/4/BVCSXAR78FEDI1...


In [849]:
my_info_columnas(data, solo_nan='no')

Tu dataframe tiene:
25.02% de valores NaN en total.
121220 filas
23 columnas:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
floor,113321,93.48,float64,183
expenses,106958,88.23,float64,983
rooms,73830,60.91,float64,32
price_usd_per_m2,52603,43.39,float64,25568
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
surface_total_in_m2,39328,32.44,float64,1688
price_per_m2,33562,27.69,float64,25186
currency,20411,16.84,object,5


In [851]:
# Vemos el tipo de propiedad:
my_columna_valores(data, col='property_type')

Columna: property_type
Cantidad de elementos únicos:  4
APARTMENT    71065
HOUSE        40268
PH            5751
STORE         4136
Name: property_type, dtype: int64


In [853]:
# Vemos la localidad:
my_columna_valores(data, col='place_name')

Columna: place_name
Cantidad de elementos únicos:  1061
CÓRDOBA                        9254
ROSARIO                        8504
MAR DEL PLATA                  6534
TIGRE                          3324
NORDELTA                       3315
                               ... 
RUFINO                            1
CAMPO GRANDE                      1
RÍO GALLEGOS                      1
FARO NORTE                        1
BARRIO CERRADO "LA DAMASIA"       1
Name: place_name, Length: 1061, dtype: int64


In [855]:
# Vemos la provincia:
my_columna_valores(data, col='state_name')

Columna: state_name
Cantidad de elementos únicos:  28
CAPITAL FEDERAL                 32316
BS.AS. G.B.A. ZONA NORTE        25560
BS.AS. G.B.A. ZONA SUR          13952
CÓRDOBA                         12069
SANTA FE                        10172
BUENOS AIRES COSTA ATLÁNTICA    10006
BS.AS. G.B.A. ZONA OESTE         9322
BUENOS AIRES INTERIOR            2291
RÍO NEGRO                         808
NEUQUÉN                           733
MENDOZA                           681
TUCUMÁN                           674
CORRIENTES                        583
MISIONES                          464
ENTRE RÍOS                        369
SALTA                             278
CHUBUT                            259
SAN LUIS                          252
LA PAMPA                          157
FORMOSA                            65
CHACO                              57
SAN JUAN                           40
TIERRA DEL FUEGO                   31
CATAMARCA                          27
JUJUY                             

---
### Imputacion de valores faltante por medio de otro campo

In [839]:
# Analizamos valores nulos:
my_info_columnas(data)

Tu dataframe tiene:
25.02% de valores NaN en total.
121220 filas
23 columnas de las cuales 18 tienen valores NaN:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
floor,113321,93.48,float64,183
expenses,106958,88.23,float64,983
rooms,73830,60.91,float64,32
price_usd_per_m2,52603,43.39,float64,25568
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
surface_total_in_m2,39328,32.44,float64,1688
price_per_m2,33562,27.69,float64,25186
price,20410,16.84,float64,9747


In [681]:
# Vemos que floor tiene muchos valores nulos.
# Veamos sus valores:
my_columna_valores(data, col='floor')

Columna: floor
Cantidad de elementos únicos:  183
NaN       113321
1.0         2080
2.0         1542
3.0          914
4.0          658
5.0          484
6.0          483
7.0          355
8.0          248
9.0          205
10.0         181
11.0         123
12.0         116
13.0          69
14.0          49
15.0          41
17.0          31
16.0          24
22.0          16
18.0          15
19.0          10
20.0           6
27.0           6
25.0           6
21.0           6
33.0           5
108.0          5
104.0          4
1609.0         4
31.0           4
403.0          4
23.0           3
36.0           3
702.0          3
106.0          3
301.0          3
113.0          3
907.0          3
24.0           3
1608.0         2
29.0           2
32.0           2
105.0          2
50.0           2
64.0           2
121.0          2
905.0          2
47.0           2
208.0          2
103.0          2
42.0           2
49.0           2
102.0          2
408.0          2
304.0          2
203.0          

In [682]:
# Vemos los registros con dato y sin dato de floor:
display(data[data.floor.isnull()].sample(5))
display(data[data.floor.notnull()].sample(5))

# Notamos que floor puede estar en title o description. Así que trataremos de obtenerlo de ahí. No está relacionada con algun otro parámetro ya que es única de cada inmueble.
display(data[['title', 'description']].sample(10))
my_columna_valores(data, col='title')
print("---------------------------------------------------------------")
my_columna_valores(data, col='description')

,property_type,place_name,place_with_parent_names,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
82903,APARTMENT,SANTOS LUGARES,|ARGENTINA|BS.AS. G.B.A. ZONA OESTE|TRES DE FE...,BS.AS. G.B.A. ZONA OESTE,3428075.0,NaN,NaN,NaN,65000.0,USD,1146892.50,65000.00,45.0,39.0,1444.444444,1666.666667,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1AZU0_VENTA_DEPART...,DEPARTAMENTO 2 AMBIENTES AL FRENTE DE 41M2 CUB...,HERMOSO 2 AMBIENTES CON PARRILLA PROPIA ENTREG...,HTTPS://THUMBS4.PROPERATI.COM/1/AWQBDKKVS_UGCZ...
115482,APARTMENT,CÓRDOBA,|ARGENTINA|CÓRDOBA|CÓRDOBA|,CÓRDOBA,3860259.0,NaN,NaN,NaN,90000.0,USD,1588005.00,90000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1CB8L_VENTA_DEPART...,"DEPARTAMENTO DE 2 DORMITORIOS , O UN DORMITORI...",DPTO 2 DORM NVA CBA U$S 90000,HTTPS://THUMBS4.PROPERATI.COM/6/DL90HH-4QXOU3H...
107409,APARTMENT,ROSARIO,|ARGENTINA|SANTA FE|ROSARIO|,SANTA FE,3838574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,NaN,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/1BY2B_VENTA_DEPART...,"DEPARTAMENTO, SANTA FE, ROSARIO",DEPARTAMENTO - ROSARIO,HTTPS://THUMBS4.PROPERATI.COM/5/LV_FYOVAJIDOAR...
38794,HOUSE,MISIONES,|ARGENTINA|MISIONES|,MISIONES,3430657.0,"-27.600086,-55.316666",-27.600086,-55.316666,1300000.0,ARS,1286438.79,72908.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/18M8N_VENTA_CASA_M...,***SE VENDE 3 CASAS INDEPENDIENTES*** $ 1.300....,***SE VENDE 3 CASAS INDEPENDIENTES*** $ 1.300....,HTTPS://THUMBS4.PROPERATI.COM/6/3UL1BOCQUAWAX4...
1543,APARTMENT,BELGRANO,|ARGENTINA|CAPITAL FEDERAL|BELGRANO|,CAPITAL FEDERAL,3436077.0,"-34.562708,-58.465375",-34.562708,-58.465375,119144.0,USD,2102236.30,119144.00,NaN,42.0,NaN,2836.761905,NaN,2.0,NaN,HTTP://WWW.PROPERATI.COM.AR/15IP7_VENTA_DEPART...,"VENTA - DEPARTAMENTO - ARGENTINA, BUENOS AIRES...",DEPARTAMENTO EN VENTA UBICADO EN BELGRANO,HTTPS://THUMBS4.PROPERATI.COM/3/1RQOSXE_VRH38M...


,property_type,place_name,place_with_parent_names,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
7025,APARTMENT,OLIVOS,|ARGENTINA|BS.AS. G.B.A. ZONA NORTE|VICENTE LÓ...,BS.AS. G.B.A. ZONA NORTE,3430310.0,NaN,NaN,NaN,735258.0,USD,12973259.78,735258.0,169.0,137.0,4350.639053,5366.846715,2.0,NaN,NaN,HTTP://WWW.PROPERATI.COM.AR/15WBP_VENTA_DEPART...,"DEPARTAMENTO VISTA, TERMINADO DESDE EL POZO. Z...",DEPARTAMENTO 169M² CON COCHERA EN DIAZ DE SOLI...,NaN
49763,PH,NEUQUÉN,|ARGENTINA|NEUQUÉN|NEUQUÉN|,NEUQUÉN,3843123.0,"-38.9513389666,-68.0664865562",-38.951339,-68.066487,1721000.0,ARS,1703047.14,96520.0,37.0,37.0,2608.648649,46513.513514,11.0,1.0,NaN,HTTP://WWW.PROPERATI.COM.AR/19CSN_VENTA_PH_NEU...,"VENTA DE DEPARTAMENTO MONOAMBIENTE EN CENTRO, ...",PH EN VENTA,HTTPS://THUMBS4.PROPERATI.COM/3/CLPO40IP_VD9_O...
16881,APARTMENT,CÓRDOBA,|ARGENTINA|CÓRDOBA|CÓRDOBA|,CÓRDOBA,3860259.0,"-31.4301782,-64.1930063",-31.430178,-64.193006,1250000.0,USD,22055625.00,1250000.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,HTTP://WWW.PROPERATI.COM.AR/16ZQ7_VENTA_EDIFIC...,DEPARTAMENTO EN VENTA DE 1 DORM. EN CORDOBA,NUEVA CBA 1 DOR CON BALCÓN NUEVO PILETA,HTTPS://THUMBS4.PROPERATI.COM/1/LHVZBZBOVZRK8V...
51448,APARTMENT,CABALLITO,|ARGENTINA|CAPITAL FEDERAL|CABALLITO|,CAPITAL FEDERAL,3435874.0,"-34.6152486,-58.4319072",-34.615249,-58.431907,240000.0,USD,4234680.00,240000.0,91.0,71.0,2637.362637,3380.281690,8.0,3.0,3700.0,HTTP://WWW.PROPERATI.COM.AR/19GYD_VENTA_DEPART...,VENTA DE DEPARTAMENTO 3 AMBIENTES EN CABALLITO...,DEPARTAMENTO EN VENTA,HTTPS://THUMBS4.PROPERATI.COM/6/R5FP4_EFL3N0_O...
61235,APARTMENT,CHACARITA,|ARGENTINA|CAPITAL FEDERAL|CHACARITA|,CAPITAL FEDERAL,3435506.0,"-34.5849804,-58.4542085",-34.584980,-58.454208,89000.0,USD,1570360.50,89000.0,37.0,34.0,2405.405405,2617.647059,9.0,1.0,NaN,HTTP://WWW.PROPERATI.COM.AR/1A53N_VENTA_DEPART...,VENTA DE DEPARTAMENTO MONOAMBIENTE DIVISIBLE E...,DEPARTAMENTO EN VENTA,HTTPS://THUMBS4.PROPERATI.COM/2/PSZRHUDTIYFPXZ...


,title,description
94236,CASA EN VALENTIN ALSINA,CASA EN ESQUINA SOBRE AMPLIO LOTE. OPORTUNIDAD...
15271,MANSIÓN CASA VILLA CONSTITUCIÓN + CASA DE CASE...,CODIGO: 2366-377 UBICADO EN: VILLA CONSTITUCIÓ...
69056,"MODERNO DEPARTAMENTO DE 1, 2 Y 3 AMB UBICADO E...","MODERNO DEPARTAMENTO DE 1, 2 Y 3 AMBIENTES UBI..."
26583,DEPA 1 AMB DIVISIBLE EN 24 CUOTAS EN PESOS,DEPTO 1 AMBIENTE TOTALMENTE DIVISIBLEAL FRENTE...
120563,CASA - SAN ISIDRO,IMPORTANTE PROPIEDAD DESARROLLADA EN UNA SOLA ...
12419,CASA EN VENTA,"CASA SEIS AMBIENTES EN VENTA EN MUÑIZ BARRIO ""..."
91580,LOCAL EN VENTA EN GONNET,CAMINO GRAL. BELGRANO Y 500IMPORTANTE LOCAL CO...
12473,DEPARTAMENTO EN VENTA,RESERVADO.EDIFICIO EN TORRE DE CATEGORÍA 4 AMB...
34991,"CASA 168M² CON COCHERA EN JURAMENTO 400, SAN I...",MUY BUEN ESTADO GENERAL HERMOSA PROPIEDAD DESA...
59696,CASA A TERMINAR EN BARRIO CERRADO. VILLA ESQUIU!,CORREDOR RESPONSABLE: JUAN MANUEL QUIROGA - CP...


Columna: title
Cantidad de elementos únicos:  71654
DEPARTAMENTO EN VENTA                                                5025
CASA EN VENTA                                                        2759
DEPARTAMENTO - NUEVA CORDOBA                                         1003
PH EN VENTA                                                           949
DEPARTAMENTO - CENTRO                                                 685
                                                                     ... 
LUMINOSO DEPTO A MTS DE LA EST DE BANFIELD                              1
DUPLEX EN VENTA A ESTRENAR APTO CREDITO                                 1
PH AL FRENTE DE 3 AMBIENTES MUY COMODO CON 2 PATIOS EN SAN MARTIN       1
AV RIVADAVIA AL 1200 // DPTO TRES AMBIENTES IMPECABLE                   1
MATIENZO 1080 - MONTE GRANDE                                            1
Name: title, Length: 71654, dtype: int64
---------------------------------------------------------------
Columna: description
Cantidad

In [623]:
NO VAAA SOLO LO DOJO COMO EJEMPLO
# Quiero ver la cantidad de nulos de floor por tipo de propiedad y luego por provincia/localidad para ver si hay alguna relación:
print(data.groupby('property_type').apply(lambda x: (x['floor'].isnull().sum()/len(x)*100).round(2)))
print("----------------------------------------------")
print(data.groupby('state_name').apply(lambda x: (x['floor'].isnull().sum()/len(x)*100).round(2)))
print("----------------------------------------------")
print(data.groupby(['state_name', 'property_type']).apply(lambda x: (x['floor'].isnull().sum()/len(x)*100).round(2)))

property_type
PH           90.04
apartment    90.82
house        98.10
store        99.03
dtype: float64
----------------------------------------------
state_name
Bs.As. G.B.A. Zona Norte         93.01
Bs.As. G.B.A. Zona Oeste         93.89
Bs.As. G.B.A. Zona Sur           98.01
Buenos Aires Costa Atlántica     98.37
Buenos Aires Interior            98.87
Capital Federal                  90.69
Catamarca                        96.30
Chaco                           100.00
Chubut                           99.61
Corrientes                      100.00
Córdoba                          89.34
Entre Ríos                       92.68
Formosa                         100.00
Jujuy                            88.46
La Pampa                        100.00
La Rioja                        100.00
Mendoza                          96.04
Misiones                         83.62
Neuquén                          88.27
Río Negro                        96.53
Salta                            88.49
San Juan          

In [624]:
 NO VA SOLO LO DEJO COMO JEEMPLO
# Vemos los de Córdoba y PH que tenen mas datos:
data[(data['state_name']=='Córdoba') & (data['property_type']=='PH')].sample(10)

,property_type,place_name,place_with_parent_names,state_name,geonames_id,lat-lon,lat,lon,price,currency,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
29853,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,40.0,NaN,NaN,4.0,2.0,NaN,http://www.properati.com.ar/17zzn_venta_ph_cor...,OCTAVIO VI - VENTA - DEPARTAMENTO - 1 DORMITOR...,PH EN VENTA,https://thumbs4.properati.com/7/4jdTH14r6im1vd...
29822,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,73.0,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/17zys_venta_ph_cor...,TERRAZAS DE O´HIGGINS 2 (TORRE 4) - VENTA - DE...,PH EN VENTA,https://thumbs4.properati.com/8/EkORCeZ8S7YD85...
29833,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,54.0,NaN,NaN,2.0,2.0,NaN,http://www.properati.com.ar/17zz3_venta_ph_cor...,TERRAZAS DE O´HIGGINS 2 (TORRE 4) - VENTA - DE...,PH EN VENTA,https://thumbs4.properati.com/2/kLMWiGxtLCmZ7w...
29965,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,49.0,NaN,NaN,7.0,2.0,NaN,http://www.properati.com.ar/1802z_venta_ph_cor...,FIUMICCINO VII - VENTA - DEPARTAMENTO - 1 DOR...,PH EN VENTA,https://thumbs4.properati.com/7/yyugldlZePCKIa...
29957,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,76.0,NaN,NaN,3.0,3.0,NaN,http://www.properati.com.ar/1802r_venta_ph_cor...,CASA BLANCA - VENTA - DEPARTAMENTO - 1 DORMIT...,PH EN VENTA,https://thumbs4.properati.com/9/75OM8wg9IPKGBY...
9732,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,"-31.335784,-64.2635034",-31.335784,-64.263503,NaN,NaN,...,200.0,NaN,NaN,2.0,4.0,NaN,http://www.properati.com.ar/165ge_venta_ph_cor...,Duplex en Venta de 2 dorm. en Cordoba,Duplex Venta en Housing Zona Norte,https://thumbs4.properati.com/3/7PHpZywN7RqUZ-...
29819,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,NaN,NaN,NaN,NaN,NaN,...,78.0,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/17zyp_venta_ph_cor...,TERRAZAS DE O´HIGGINS 2 (TORRE 4) - VENTA - DE...,PH EN VENTA,https://thumbs4.properati.com/4/LC25YLBbhF4jHC...
101508,PH,Punilla,|Argentina|Córdoba|Punilla|,Córdoba,3839996.0,NaN,NaN,NaN,1250000.0,ARS,...,66.0,1062.190758,18939.393939,NaN,NaN,NaN,http://www.properati.com.ar/1bpgi_venta_ph_pun...,"Hermosa casita ph en barrio la quinta, a pocas...",PH - Villa Carlos Paz,https://thumbs4.properati.com/5/f3_QFsUZVWEZc6...
47021,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Córdoba,3860259.0,"-31.3532518,-64.2209187",-31.353252,-64.220919,160000.0,USD,...,140.0,NaN,1142.857143,2.0,3.0,NaN,http://www.properati.com.ar/197e0_venta_ph_cor...,Duplex en Venta de 3 dorm. en Cordoba,"Duplex 3 Dor, Vta. Chacras del Norte",https://thumbs4.properati.com/5/By_hT32BkZDtqw...
16181,PH,Punilla,|Argentina|Córdoba|Punilla|,Córdoba,3839996.0,"-31.4317003,-64.5113037",-31.431700,-64.511304,850000.0,ARS,...,50.0,953.422400,17000.000000,NaN,NaN,300.0,http://www.properati.com.ar/16xs2_venta_ph_pun...,CODIGO: 1178-VTA030 ubicado en: PERITO MORENO ...,DEPARTAMENTO AMPLIO EN PLAYAS DE ORO,https://thumbs4.properati.com/4/4M7IyMWCNzy0kB...


---
### Diagrama de expensas para los valores existentes.

Si bien las expensas tienen valores faltantes, en función de la provincia y tipo de casa nos puede dar una idea de los impuestos en dicha categoría.

In [843]:
# Analicemos expenses:
# Podríamos analizar las expensas de solo los elementos que tienene el dato para sacar alguna inferencia en general.
my_info_columnas(data)
my_columna_valores(data, col='expenses')

Tu dataframe tiene:
25.02% de valores NaN en total.
121220 filas
23 columnas de las cuales 18 tienen valores NaN:


,Cantidad de NaN,% de NaN,Dtype,Valores Unicos
floor,113321,93.48,float64,183
expenses,106958,88.23,float64,983
rooms,73830,60.91,float64,32
price_usd_per_m2,52603,43.39,float64,25568
lat-lon,51550,42.53,object,47204
lat,51550,42.53,float64,46496
lon,51550,42.53,float64,46523
surface_total_in_m2,39328,32.44,float64,1688
price_per_m2,33562,27.69,float64,25186
price,20410,16.84,float64,9747


Columna: expenses
Cantidad de elementos únicos:  983
NaN        106958
1.0          1028
1000.0        627
1500.0        617
2000.0        561
            ...  
1935.0          1
22762.0         1
2472.0          1
1883.0          1
11920.0         1
Name: expenses, Length: 983, dtype: int64


In [628]:
print(data.groupby(['state_name', 'property_type']).apply(lambda x: (x['expenses'].isnull().sum()/len(x)*100).round(2)))

state_name                property_type
Bs.As. G.B.A. Zona Norte  PH                95.63
                          apartment         84.75
                          house             89.94
                          store             87.44
Bs.As. G.B.A. Zona Oeste  PH                95.94
                                            ...  
Tierra Del Fuego          store            100.00
Tucumán                   PH                75.00
                          apartment         86.35
                          house             87.38
                          store             87.10
Length: 98, dtype: float64
